# WORD EMBEDDING

## 参考资料

[什么是 word embedding?](https://www.zhihu.com/question/32275069/answer/61059440)

[WORD EMBEDDINGS: ENCODING LEXICAL SEMANTICS](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#sphx-glr-beginner-nlp-word-embeddings-tutorial-py)

[YJango的Word Embedding--介绍](https://zhuanlan.zhihu.com/p/27830489)

## 概念解析

## 代码解析

与我们制作独热向量（one-hot vectors）时候为每个单词定义一个唯一索引类似，在使用词嵌入的时候我们也需要为每个单词定义索引，方便查找。

词嵌入被储存为了一个 $ |V|×D $ 的矩阵，其中 $ D $ 是嵌入的维度，所以某单词的索引为 $ i $，那么它就被嵌入在矩阵的第 $ i $ 行中。在代码中，从单词到索引的映射是一个名叫 word_to_ix 的字典。

在 PyTorch 中，torch.nn.Embedding 可以完成词嵌入功能，包含两个参数：词表大小和嵌入维度。

使用 torch.LongTensor 进行索引（这是一个 64-bit integer，带符号数）。

In [27]:
# Author: Robert Guthrie
# 补充：Dr_David_S

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)  # 设定随机种子，返回一个torch.Generator对象

现在我们设置一个字典，用最简单的 hello world 做例子：

In [28]:
word_to_ix = {"hello": 0, "world": 1}
word_to_ix

{'hello': 0, 'world': 1}

使用 torch.nn.Embedding 方法准备进行词嵌入，参数的意义是：有2个词语，5个维度。

In [29]:
embeds = nn.Embedding(num_embeddings=2, embedding_dim=5)  # 2 words in vocab, 5 dimensional embeddings
embeds

Embedding(2, 5)

现在将 hello 对应的值 0 转换为一个 longtensor 类型,两种方法都行。

In [30]:
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
lookup_tensor

tensor([0])

In [31]:
lookup_tensor_test = torch.LongTensor([word_to_ix["hello"]])
lookup_tensor_test

tensor([0])

现在使用之前定义的 embeds 模型对 hello 这个次进行词嵌入（或者说转换为词向量）：

In [32]:
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


原来的代码到此就结束了，但是这里我们希望再试试对 world 的词嵌入：

In [33]:
lookup_tensor2 = torch.tensor([word_to_ix["world"]], dtype=torch.long)
lookup_tensor2

tensor([1])

In [34]:
world_embed = embeds(lookup_tensor2)
print(world_embed)

tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]],
       grad_fn=<EmbeddingBackward>)


从上述结果来看，似乎对于 hello 的 embedding 没有涉及到 world？

事实上，embeds只是一个随机生成的矩阵，目前其中的向量并不能代表任何意义，如下：

In [35]:
embeds.weight

Parameter containing:
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519],
        [-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]], requires_grad=True)

>nn.Embedding.weight初始化分布符合标准正态分布 $ N(0,1) $，即均值 $\mu=0$，方差 $\sigma=1$ 的正态分布。
>
>因为这里的初始化参数只设定了两个单词，所以其中：
>
>某一行代表单词 'hello'
>
>另一行代表单词 'world'

## 一个 N-Gram 语言模型

之前我们的 embeds 矩阵的初始值是随机生成的，现在我们将要计算一些训练样例的损失函数，然后使用反向传播去更新参数。

### 文本处理

首先设定参数：

In [36]:
CONTEXT_SIZE = 2  # 指上下文的size
EMBEDDING_DIM = 10

其中，上下文是指目标单词的前 $ n $ 个单词，或者后 $ n $ 个单词，或者前后都有总共 $ n $ 个单词。

在这里，CONTEXT_SIZE 指的是上文的两个单词，意味着如果以后遇到上下文的情况，需要乘以二。

EMBEDDING_DIM 指的是会将其 embedding 为十维向量。

在本例中，我们使用的是莎士比亚的十四行诗，使用空格分词：

In [37]:
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

>SONNET 2：
>
>当四十个冬天围攻你的朱颜，
>
>在你美的园地挖下深的战壕，
>
>你青春的华服，那么被人艳羡，
>
>将成褴褛的败絮，谁也不要瞧：
>
>那时人若问起你的美在何处，
>
>哪里是你那少壮年华的宝藏，
>
>你说，"在我这双深陷的眼眶里，
>
>是贪婪的羞耻，和无益的颂扬。"
>
>你的美的用途会更值得赞美，
>
>如果你能够说，"我这宁馨小童
>
>将总结我的账，宽恕我的老迈，"
>
>证实他的美在继承你的血统！
>
>这将使你在衰老的暮年更生，
>
>并使你垂冷的血液感到重温。
>

In [38]:
test_sentence

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep',
 'trenches',
 'in',
 'thy',
 "beauty's",
 'field,',
 'Thy',
 "youth's",
 'proud',
 'livery',
 'so',
 'gazed',
 'on',
 'now,',
 'Will',
 'be',
 'a',
 "totter'd",
 'weed',
 'of',
 'small',
 'worth',
 'held:',
 'Then',
 'being',
 'asked,',
 'where',
 'all',
 'thy',
 'beauty',
 'lies,',
 'Where',
 'all',
 'the',
 'treasure',
 'of',
 'thy',
 'lusty',
 'days;',
 'To',
 'say,',
 'within',
 'thine',
 'own',
 'deep',
 'sunken',
 'eyes,',
 'Were',
 'an',
 'all-eating',
 'shame,',
 'and',
 'thriftless',
 'praise.',
 'How',
 'much',
 'more',
 'praise',
 "deserv'd",
 'thy',
 "beauty's",
 'use,',
 'If',
 'thou',
 'couldst',
 'answer',
 "'This",
 'fair',
 'child',
 'of',
 'mine',
 'Shall',
 'sum',
 'my',
 'count,',
 'and',
 'make',
 'my',
 'old',
 "excuse,'",
 'Proving',
 'his',
 'beauty',
 'by',
 'succession',
 'thine!',
 'This',
 'were',
 'to',
 'be',
 'new',
 'made',
 'when',
 'thou',
 'art',
 'old,',
 

按道理，我们在训练之前输入的数据应当是有标记的，我们暂时忽略这个事情（后面会说明）。

然后我们要建立一个元组（tuple），元组的格式为：

$$ ([word_{i-2}, word_{i-1}], word_{i}) $$

其中 $word_{i}$ 就是目标词 $ target word $ 

根据以上方法遍历整首诗，然后生成多个元组，放入列表 trigrams 中。

In [39]:
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


set() 函数创建一个无序不重复元素集，可以剔除掉重复的元素，同时可能可以在部分模型如 RNN 中打乱文档上下文，忽略掉 RNN 权重偏向后输入的词语的缺点。

In [40]:
vocab = set(test_sentence)  # 去重
word_to_ix = {word: i for i, word in enumerate(vocab)}

### 搭建网络

接下来我们构建一个网络，名叫 NGramLanguageModeler：

In [41]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)  # 这步实现了embedding
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

从上述网络来看，init 中存在三个层，分别是：

- 词嵌入层（embedding层）
- 第一个全连接层（linear1）
- 第二个全连接层（linear2）

算上前向传播的过程就是：

- 词嵌入层
- 转换为一维
- 第一个全连接层
- ReLU激活
- 第二个全连接层
- log_softmax输出

### 设置损失函数和优化函数

- 使用NLLLoss()，这个函数其实就是一个不带 log_softmax 的交叉熵损失函数。

- 使用optim.SGD()，最传统的随机梯度下降，没什么好说的。

需要注意的是，model 的输入参数有三个，第一个就是去重后的上下文三词元组数量，第二三个则是我们设定好的 CONTEXT_SIZE 和 EMBEDDING_DIM。

In [42]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

### 训练模型

注意：
```Python
for context, target in trigrams
```
这句话指的是我们的训练样本是上下文（其实只有上文），标签是当前词。这也是为什么之前没有标记的原因。

In [43]:
trigrams

[(['When', 'forty'], 'winters'),
 (['forty', 'winters'], 'shall'),
 (['winters', 'shall'], 'besiege'),
 (['shall', 'besiege'], 'thy'),
 (['besiege', 'thy'], 'brow,'),
 (['thy', 'brow,'], 'And'),
 (['brow,', 'And'], 'dig'),
 (['And', 'dig'], 'deep'),
 (['dig', 'deep'], 'trenches'),
 (['deep', 'trenches'], 'in'),
 (['trenches', 'in'], 'thy'),
 (['in', 'thy'], "beauty's"),
 (['thy', "beauty's"], 'field,'),
 (["beauty's", 'field,'], 'Thy'),
 (['field,', 'Thy'], "youth's"),
 (['Thy', "youth's"], 'proud'),
 (["youth's", 'proud'], 'livery'),
 (['proud', 'livery'], 'so'),
 (['livery', 'so'], 'gazed'),
 (['so', 'gazed'], 'on'),
 (['gazed', 'on'], 'now,'),
 (['on', 'now,'], 'Will'),
 (['now,', 'Will'], 'be'),
 (['Will', 'be'], 'a'),
 (['be', 'a'], "totter'd"),
 (['a', "totter'd"], 'weed'),
 (["totter'd", 'weed'], 'of'),
 (['weed', 'of'], 'small'),
 (['of', 'small'], 'worth'),
 (['small', 'worth'], 'held:'),
 (['worth', 'held:'], 'Then'),
 (['held:', 'Then'], 'being'),
 (['Then', 'being'], 'asked

In [44]:
for epoch in range(100):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        # 这一步把上下文单词转换为字典中对应单词的值，作为tensor
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[519.3395328521729, 516.7286524772644, 514.1358370780945, 511.56134843826294, 509.0043873786926, 506.4630753993988, 503.9358084201813, 501.4213182926178, 498.91916370391846, 496.4259123802185, 493.94328260421753, 491.46931982040405, 489.00418972969055, 486.5483543872833, 484.10179924964905, 481.6624879837036, 479.23038053512573, 476.80602836608887, 474.3874454498291, 471.97468400001526, 469.56671595573425, 467.16306471824646, 464.76284623146057, 462.3669693470001, 459.9745342731476, 457.58553290367126, 455.199177980423, 452.81409525871277, 450.4298937320709, 448.044686794281, 445.6583981513977, 443.2708628177643, 440.883748292923, 438.49316215515137, 436.09905529022217, 433.70160484313965, 431.29941725730896, 428.8915560245514, 426.47898030281067, 424.0606231689453, 421.63829016685486, 419.2113003730774, 416.7768678665161, 414.3365406990051, 411.8876028060913, 409.4320206642151, 406.96857619285583, 404.4972274303436, 402.01919746398926, 399.5325789451599, 397.0351777076721, 394.5294361

现在我们得到了一个损失逐渐降低的模型，可以看出这个模型的结果是收敛的，如果继续训练下去，可能会更准确，但是到此为止吧。

## 练习：计算词嵌入：CBOW模型

连续词袋模型（Continuous Bag-of-Words model ），又被称为CBOW模型，经常用于自然语言处理深度学习。

CBOW模型使用一段文本的中间词作为目标词，即给出目标词的上下文单词作为训练数据。CBOW去除了上下文各词语的词序信息，使用的是上下文各词的词向量的平均值。

CBOW对于一段长度为 $ n $ 的训练样本：$ w_{i-(n-1)}, ...,w_i $ ，其输入为: $$ x = \frac {1}{n - 1}\sum_{w_j \in c}e(w_j) $$

其中 $ w $ 是目标词，即 $ w_i $， $ c $ 是上下文，上式就是将输入求平均。

故CBOW模型根据上下文的表示，直接对目标词进行预测： $$ P(w|c) = \frac{\exp(e'(w)^Tx)}{\sum_{w'\in V}\exp \lgroup e'(w')^Tx\rgroup} $$ 

CBOW的目标是最大化 $$ \sum_{(w,c)\in D}\log P(w|c) $$

其中 $D$ 指的是整段语料（或者说整篇文章）。

要在PyTorch中实现这个模型，需要注意的是：

- 考虑一下需要定义哪些参数
- 确保你清楚每一步需要哪些形状的矩阵，使用 .view() 函数来 reshape 矩阵

### 1.分割语料

代码示例如下，第一步，先按空格分隔语料中的单词，其中 2 表示上文两个单词加下文两个单词。

In [45]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

使用 set() 函数将分词唯一化并打乱顺序，然后建立一个名叫 word_to_ix 的字典，字典的 key 是单词，value 是单词的索引。

接下来使用data列表来储存文本和目标了，把第 $i$ 个单词作为目标，把第$[i-2, i-1, i+1, i+2]$ 个单词作为上下文（特征），注意这里使用的是有序的语料。

In [56]:
# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    # context保存目标单词上下文，分别2个单词
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    # target保存目标单词
    target = raw_text[i]
    # 存入data列表中
    data.append((context, target))
    
print(data[:5])
print(len(data))
print(vocab_size)

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
58
49


可以看出我们的上下文目标词组有58个，但是非重复单词有49个。

### 2.建立CBOW类

建立 CBOW 类，由于本节有习题性质，CBOW 类并没有内容，需要自己填空。

In [57]:
class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    """从字典中抽取word对应的value
    
    输入：
    context:上下文列表（四个单词）
    word_to_ix:单词的字典
    
    输出：context中的单词对应的longtensor
    """
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


first_context = make_context_vector(data[0][0], word_to_ix)  # example
print(first_context)

tensor([ 2, 33, 46, 19])


这里给了一个例子，就是

```Python
make_context_vector(data[0][0], word_to_ix)  # example
```
简单分析一下：

```Python
data[0][0]
```
指的就是前面 data 列表中第一个tuple中的第一个元素，即:

In [58]:
print(data[0][0])

['We', 'are', 'to', 'study']


上述四个单词在字典 word_to_ix 中对应的值如下：

In [59]:
print(word_to_ix['We'])
print(word_to_ix['are'])
print(word_to_ix['to'])
print(word_to_ix['study'])

2
33
46
19


故其组成的 longtensor 自然就是 tensor([20, 27, 26, 21]) 啦~

### 简单embedding演示

现在，使用 torch.nn.Embedding 方法建立一个embedding模型，目标是去重后的49个词，每个词20维。

In [60]:
embeds = nn.Embedding(num_embeddings=49, embedding_dim=20)  # 58 words in vocab, 20 dimensional embeddings
embeds

Embedding(49, 20)

我们尝试使用上面的 embeds 模型将 first_context 向量化：

In [61]:
first_context_embed = embeds(first_context)
print(first_context_embed)

tensor([[-2.1191, -0.6800, -0.7205,  1.7116,  0.5737,  0.3592,  0.1365, -0.9688,
         -0.6356, -1.0128, -0.1175,  0.4506, -1.0744, -0.2628, -2.5613,  0.3344,
          0.1283, -0.7623,  1.3541, -0.5149],
        [ 0.7784,  1.2752,  0.4514,  0.3024, -0.1240,  0.9851, -0.4329, -0.0269,
         -0.6201, -0.0385,  0.8849, -0.2759, -0.0669, -0.2744, -0.1082, -1.7570,
         -1.0108, -1.3906, -0.9470,  0.8091],
        [-0.7886, -0.6210,  1.0869, -0.0139, -0.0809, -0.4313,  0.6058,  2.0395,
         -0.2878,  1.1277, -1.8568,  0.2506,  0.7219,  1.3674,  0.0862, -1.0147,
          0.0826, -0.2459, -1.1079,  0.0940],
        [ 0.9016, -1.3125, -0.7962,  0.4715, -0.6822, -0.4986,  0.8168,  1.4585,
         -0.5664,  1.1618, -1.7296,  0.6157,  1.3086, -0.8415, -1.8773, -0.6930,
         -0.4665, -1.0278,  0.8361, -0.0702]], grad_fn=<EmbeddingBackward>)


可以看出已经成功的完成了我们向量化的第一步实验，接下来的任务就是把所有的上下文和目标单词向量化，然后建立一个 CBOW 模型对这些向量进行训练。

顺便看看 embeds 的初始化，应当是随机初始化了一个符合标准正态分布 $ N(0,1) $的 49 行，20 列的矩阵。

In [62]:
embeds.weight

Parameter containing:
tensor([[-1.1007e+00, -1.9407e-01,  2.2328e+00,  9.0314e-01,  3.3703e-01,
         -5.7499e-02, -1.1944e-02,  1.5079e+00,  5.7785e-01,  3.6901e-01,
         -1.6980e+00, -1.8248e+00,  9.5325e-03,  1.5280e-01,  2.8693e+00,
          4.2386e-01,  9.1356e-01, -8.1239e-01, -8.8108e-01, -2.6689e-01],
        [-2.2471e-01,  3.4927e-01, -4.1475e-01,  1.1367e+00,  5.2791e-01,
         -1.2952e+00,  4.2541e-01,  5.0982e-01, -9.8929e-01, -9.2530e-01,
         -5.3013e-01, -8.1304e-01,  8.5257e-01,  9.1699e-01, -1.1232e+00,
          1.3981e+00,  2.5184e-01,  9.6266e-01, -2.6745e-01, -1.2740e+00],
        [-2.1191e+00, -6.8000e-01, -7.2047e-01,  1.7116e+00,  5.7365e-01,
          3.5923e-01,  1.3653e-01, -9.6875e-01, -6.3558e-01, -1.0128e+00,
         -1.1747e-01,  4.5057e-01, -1.0744e+00, -2.6283e-01, -2.5613e+00,
          3.3438e-01,  1.2826e-01, -7.6229e-01,  1.3541e+00, -5.1493e-01],
        [-6.7641e-01,  5.9749e-01,  2.2694e-01, -2.1956e-01,  1.6207e-01,
         -3.5

这里只是说明一下，而后我们将补全之前的 CBOW 类代码，遍历 data 中的每个元素进行embedding。

### 补全CBOW类

事实上，我们对目标词和其上下文的 embedding 是在 CBOW 类中完成的。我们基于之前的示例代码进行修改。

In [64]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        """
        层搭建
        
        输入：
        vocab_size：去重后的单词量，本例为49个
        embedding_dim：需要 embedding 的维度，本例设为20维
        context_size：上下文的大小，注意和前个例子不一样，前面是上文，这里是上下文
        """
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * 2 * embedding_dim, 128)  # 这里乘以2应该是上下文的原因
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        """
        前向传播
        
        输入：
        inputs：
        """
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)  # 输出概率
        return log_probs  # 结果

### 设置损失函数

同前个例子

In [65]:
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

### 开始训练

In [66]:
for epoch in range(100):
    total_loss = 0
    for context, target in data:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        # 这一步把上下文单词转换为字典中对应单词的值，作为tensor
        
        context_idxs = make_context_vector(context=context, word_to_ix=word_to_ix)
        # context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        # 梯度归零
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        # 输出概率
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        # 这里target其实是一个数字标签，通过数字标签在字典中倒推回单词。
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[230.33530521392822, 228.7504334449768, 227.17804646492004, 225.61784315109253, 224.0681312084198, 222.5295741558075, 221.00004506111145, 219.4789161682129, 217.96699833869934, 216.46349573135376, 214.9683482646942, 213.48054933547974, 211.99723529815674, 210.51952934265137, 209.04802346229553, 207.58251404762268, 206.12100291252136, 204.665030002594, 203.21184468269348, 201.7628242969513, 200.31635522842407, 198.874680519104, 197.43714237213135, 196.0014204978943, 194.56873106956482, 193.13860893249512, 191.71323466300964, 190.28981137275696, 188.8660478591919, 187.44388890266418, 186.0226662158966, 184.60402393341064, 183.18601298332214, 181.77017188072205, 180.35698294639587, 178.9464042186737, 177.53524804115295, 176.12641096115112, 174.7192952632904, 173.31366968154907, 171.9082760810852, 170.50400733947754, 169.10145592689514, 167.69923210144043, 166.29914903640747, 164.89859104156494, 163.50192832946777, 162.1072130203247, 160.7137155532837, 159.32071828842163, 157.9308779239654

In [67]:
len(losses)

100